---
title: "Med-Exp-Analysize"
format: 
  html:
    code-fold: true
jupyter: python3
---

对符合分析条件后的样本，按照文章需要进行分析。

In [ ]:
import pandas as pd

# 导入 Excel 文件
file_path = "C:/Users/asus/Desktop/test/stata/prepare.xlsx"
output_path = "C:/Users/asus/Desktop/test/stata/data/merge-data.xlsx"
final_output_path = "C:/Users/asus/Desktop/test/stata/data/cleaned-merge-data.xlsx"
sheet_names = ["2018", "2019", "2020", "2021", "2022", "2023"]

# 读取所有 sheet 的数据
sheets_data = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheet_names}

# 获取每个 sheet 的列名
sheets_columns = {sheet: set(data.columns) for sheet, data in sheets_data.items()}

# 找出所有 sheet 的共同变量
common_columns = set.intersection(*sheets_columns.values())
common_columns = list(common_columns)  # 保持原始顺序

# 修改这部分代码以避免 SettingWithCopyWarning
processed_sheets = []
for sheet, data in sheets_data.items():
    # 创建新的 DataFrame 而不是视图
    sheet_data = data[common_columns].copy()
    sheet_data.loc[:, 'year'] = sheet
    processed_sheets.append(sheet_data)

# 合并所有 sheet 的数据
merged_data = pd.concat(processed_sheets, ignore_index=True)

# 输出合并后的数据到指定路径
merged_data.to_excel(output_path, index=False)

# 重新导入合并后的数据
merged_data = pd.read_excel(output_path)

# 剔除键值全部为 null 或 0 的变量，同时保持原始变量的顺序
non_null_columns = merged_data.dropna(axis=1, how='all').columns
non_zero_columns = merged_data.loc[:, (merged_data != 0).any(axis=0)].columns
valid_columns = [col for col in merged_data.columns if col in non_null_columns and col in non_zero_columns]

cleaned_data = merged_data.loc[:, valid_columns]

# 输出清理后的数据到指定路径
cleaned_data.to_excel(final_output_path, index=False)

print(f"清理后的数据已输出到 {final_output_path}")

# 展示部分数据，设置随机种子以确保可重复性
sample_data = cleaned_data.sample(n=10, random_state=42)
print(sample_data)

C:\Users\asus\AppData\Local\Temp\ipykernel_23240\3104936764.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheets_data[sheet]['year'] = sheet
C:\Users\asus\AppData\Local\Temp\ipykernel_23240\3104936764.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheets_data[sheet]['year'] = sheet
C:\Users\asus\AppData\Local\Temp\ipykernel_23240\3104936764.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



清理后的数据已输出到 C:/Users/asus/Desktop/test/stata/data/cleaned-merge-data.xlsx
       3.4麻醉费  国籍  术前住院天数2  次数                4手术时间  3.1非手术治疗项目费  \
10556    19.0  中国      NaN   1                  NaN         25.0   
3602      0.0  中国      NaN   1                  NaN         24.8   
26405   315.0  中国      NaN   1                  NaN       4693.6   
36012   315.0  中国      1.0   2                  NaN         44.7   
37445   668.0  中国      NaN   1                  NaN        241.9   
47770     0.0  中国      0.0   1  2023-11-20 12:25:00       3051.9   
40811     0.0  中国      NaN   1                  NaN       6315.2   
3457      0.0  中国      NaN   1                  NaN       1366.6   
40726     0.0  中国      NaN   1                  NaN         20.1   
19409  1150.0  中国      NaN   1                  NaN        582.2   

                    4手术名称 死亡患者尸检               3手术名称     总药品费  ...  1愈合  \
10556                 NaN    NaN                 NaN   152.98  ...    甲   
3602                  NaN   

160个变量，变量还是太多了，试着把含有超过一半缺失值的变量也剔除出去，并且把入院年份和出院年份的年份提取出来，并与year变量做比较，如果入院年份和出院年份通year不一致则剔除此样本，载入合并后的数据进行处理